In [1]:
import torch
import numpy as np

In [ ]:
# Randomly generate a cos_matrix with elements in [-1, 0, 1/4, -1/4, -1/2, 1/2] with shape (3, 4)
cos_matrix = torch.randint(-2, 3, (5, 5), device="cuda").float() / 4
cos_matrix = torch.tensor([
    [1, 0, 0.5, 0.25, -0.25],
    [0, 1, -0.5, -0.25, 0.25],
    [0.5, -0.5, 1, 0, 0],
    [0.25, -0.25, 0, 1, 0],
    [-0.25, 0.25, 0, 0, 1],
    device="cuda"
])

tensor([[ 0.0000,  0.0000,  0.5000,  0.2500, -0.2500],
        [-0.5000,  0.2500,  0.2500, -0.2500, -0.2500],
        [ 0.2500,  0.0000, -0.5000, -0.2500,  0.5000],
        [ 0.2500,  0.5000, -0.2500,  0.5000, -0.5000],
        [ 0.5000, -0.5000, -0.2500, -0.2500, -0.2500]], device='cuda:0')

In [12]:
board = torch.zeros((3, 5), dtype=torch.bool, device='cuda', requires_grad=False)
actions1 = torch.tensor([0, 2, 1], dtype=torch.int, device='cuda', requires_grad=False)
actions2 = torch.tensor([1, 3, 2], dtype=torch.int, device='cuda', requires_grad=False)

In [13]:
board[torch.arange(3), actions1] = True
board[torch.arange(3), actions2] = True

In [15]:
board

tensor([[ True,  True, False, False, False],
        [False, False,  True,  True, False],
        [False,  True,  True, False, False]], device='cuda:0')

In [19]:
# state = torch.zeros((3, 5, 5), dtype=torch.float, device='cuda', requires_grad=False)

In [22]:
selected_indices = board[0].nonzero().flatten()
selected_indices

tensor([0, 1], device='cuda:0')

In [29]:
cos_arr = cos_matrix[selected_indices[0].item()]
cos_matrix = cos_arr[selected_indices][:, selected_indices]

IndexError: too many indices for tensor of dimension 1

In [31]:
cos_arr[selected_indices]

tensor([0., 0.], device='cuda:0')

In [19]:
board = torch.tensor([
    [True, False, False],
    [False, True, False],
    [True, False, True],
    [False, False, False],
])

In [20]:
adj = torch.tensor([
    [0, 1, 0],
    [1, 0, 1],
    [0, 1, 0],
], dtype=torch.bool)

In [21]:
expanded_adj = adj.unsqueeze(0).expand(4, -1, -1)
expanded_adj

tensor([[[False,  True, False],
         [ True, False,  True],
         [False,  True, False]],

        [[False,  True, False],
         [ True, False,  True],
         [False,  True, False]],

        [[False,  True, False],
         [ True, False,  True],
         [False,  True, False]],

        [[False,  True, False],
         [ True, False,  True],
         [False,  True, False]]])

In [22]:
adjacent = torch.any(expanded_adj & board[:, None, :], dim=-1)

In [23]:
adjacent

tensor([[False,  True, False],
        [ True, False,  True],
        [False,  True, False],
        [False, False, False]])

In [24]:
adjacent = adjacent & ~board
adjacent

tensor([[False,  True, False],
        [ True, False,  True],
        [False,  True, False],
        [False, False, False]])

In [25]:
print(board)
print(adjacent)

tensor([[ True, False, False],
        [False,  True, False],
        [ True, False,  True],
        [False, False, False]])
tensor([[False,  True, False],
        [ True, False,  True],
        [False,  True, False],
        [False, False, False]])


In [26]:
torch.logical_or(board, adjacent)

tensor([[ True,  True, False],
        [ True,  True,  True],
        [ True,  True,  True],
        [False, False, False]])

In [28]:
board = torch.tensor([
    [True, False, False],
])
adj = torch.tensor([
    [0, 1, 0],
    [1, 0, 1],
    [0, 1, 0],
], dtype=torch.bool)
torch.any(adj & board[:, None, :], dim=-1)

tensor([[False,  True, False]])

In [33]:
print(board[:, None, :].shape)
expanded_adj = adj.unsqueeze(0).expand(3, -1, -1)
print(expanded_adj.shape)


torch.Size([1, 1, 3])
torch.Size([3, 3, 3])


In [34]:
adj & board[:, None, :]

tensor([[[False, False, False],
         [ True, False, False],
         [False, False, False]]])

: 

In [1]:
import torch

In [24]:
adj = torch.tensor([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
], dtype=torch.bool)
board = torch.tensor([
    [1, 0, 0, 0],
    [0, 0, 1, 0]
], dtype=torch.bool)

In [25]:
# 计算每个图中与已选节点相连的节点
connected_nodes = (board.to(torch.int8) @ adj.to(torch.int8)) > 0  # (parallel_env, N)

# 计算不与已选节点相连的所有节点
non_adjacent_nodes = ~(connected_nodes | board)  # 取补集，同时排除已选节点

In [26]:
non_adjacent_nodes

tensor([[False, False,  True,  True],
        [ True,  True, False, False]])

In [23]:
connected_nodes = (board @ adj) > 0
connected_nodes

tensor([[False,  True, False, False],
        [False, False, False,  True]])

In [21]:
(connected_nodes | board)

tensor([[1, 1, 0, 0],
        [0, 0, 1, 1]])

In [19]:
temp = (connected_nodes | board)
temp = temp.to(torch.bool)
temp

tensor([[ True,  True, False, False],
        [False, False,  True,  True]])

In [20]:
~temp

tensor([[False, False,  True,  True],
        [ True,  True, False, False]])

In [16]:
~ (connected_nodes | board)

tensor([[-2, -2, -1, -1],
        [-1, -1, -2, -2]])

In [52]:
all_vectors = torch.tensor([
    [1, 2, 3],
    [3, 0, 1],
    [2, 1, 0],
    [3, 2, 1],
], dtype=torch.float16, device='cuda')
all_vectors.shape
N = all_vectors.shape[0]
d = all_vectors.shape[1]
parallel_envs = 2

In [69]:
cos_matrix = all_vectors @ all_vectors.T
cos_matrix

tensor([[14.,  6.,  4., 10.],
        [ 6., 10.,  6., 10.],
        [ 4.,  6.,  5.,  8.],
        [10., 10.,  8., 14.]], device='cuda:0', dtype=torch.float16)

In [70]:
valid_ele = torch.tensor([10, 6, 5, 8, 14], dtype=torch.float16, device='cuda')

In [58]:
boards = torch.randint(0, 2, (parallel_envs, N), dtype=torch.bool, device='cuda')
board

tensor([[False, False,  True, False],
        [False,  True,  True,  True]], device='cuda:0')

In [61]:
board = boards[0]
board

tensor([False,  True,  True,  True], device='cuda:0')

In [67]:
selected_vectors = all_vectors[board]
unselected_vectors = all_vectors[~board]

In [71]:
cos_matrix = unselected_vectors @ selected_vectors.T

In [72]:
cos_matrix

tensor([[ 6.,  4., 10.]], device='cuda:0', dtype=torch.float16)

In [75]:
def find_valid_rows(cos_matrix, valid_cos_arr, tolerance=1e-5):
    # 将valid_cos_arr扩展为与cos_matrix匹配的形状，以便进行广播比较
    expanded_valid = valid_cos_arr.unsqueeze(0).expand(cos_matrix.shape[0], -1)
    
    # 对每个元素，检查是否与valid_cos_arr中的任何值接近
    # 使用外积比较，得到shape为(cos_matrix.shape[0], cos_matrix.shape[1], valid_cos_arr.shape[0])的布尔张量
    is_close = torch.abs(cos_matrix.unsqueeze(-1) - valid_cos_arr) < tolerance
    
    # 对最后一个维度取或运算，检查每个元素是否至少与一个valid值接近
    has_valid_value = torch.any(is_close, dim=-1)
    
    # 对第二个维度取与运算，确保行中的所有元素都是有效的
    valid_rows = torch.all(has_valid_value, dim=1)
    
    return valid_rows

# 使用示例：
# valid_rows = find_valid_rows(cos_matrix, valid_cos_arr)
# filtered_cos_matrix = cos_matrix[valid_rows]

In [88]:
cos_matrix = torch.tensor([
    [10, 6, 5, 8],
    [2, 3, 4, 5],
    [3, 4, 5, 6],
    [4, 5, 6, 7],
], device='cuda', dtype=torch.float16)

In [86]:
valid_ele

tensor([10.,  6.,  5.,  8., 14.], device='cuda:0', dtype=torch.float16)

In [93]:
is_close = torch.abs(cos_matrix.unsqueeze(-1) - valid_ele) < 1e-6
has_valid_value = torch.any(is_close, dim=-1)
has_valid_value

tensor([[ True,  True,  True,  True],
        [False, False, False,  True],
        [False, False,  True,  True],
        [False,  True,  True, False]], device='cuda:0')

In [87]:
cos_matrix.unsqueeze(-1) - valid_ele

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [83]:
is_close = torch.abs(cos_matrix.unsqueeze(-1) - valid_ele) < 1e-6
is_close

tensor([[[False,  True, False, False, False],
         [False, False, False, False, False],
         [ True, False, False, False, False]]], device='cuda:0')

In [84]:
has_valid_value = torch.any(is_close, dim=-1)
has_valid_value

tensor([[ True, False,  True]], device='cuda:0')

In [98]:
torch.where(board == 0)[0]

tensor([0], device='cuda:0')

: 

In [1]:
import torch

In [ ]:
valid_cos_arr = torch.tensor([10, 6, 5, 8, 14], dtype=torch.float32)
def find_valid_rows(self, cos_matrix, valid_cos_arr, tolerance=1e-5):
    # 使用广播和向量化操作
    diff = torch.abs(cos_matrix.unsqueeze(-1) - valid_cos_arr)
    is_valid = torch.any(diff < tolerance, dim=-1)
    return torch.all(is_valid, dim=-1)

In [5]:
parallel_envs = 2
all_vectors = torch.arange(10*3).reshape(10, 3)
board = torch.zeros((parallel_envs, 10), dtype=torch.bool)
legal_actions = torch.ones((parallel_envs, 10), dtype=torch.bool)
illegal_actions = torch.zeros((parallel_envs, 10), dtype=torch.bool)


In [8]:
board[0, 0] = True
board[0, 2] = True
board[1, 1] = True
board[1, 3] = True
board

tensor([[ True, False,  True, False, False, False, False, False, False, False],
        [False,  True, False,  True, False, False, False, False, False, False]])

In [ ]:
legal_actions

In [1]:
import numpy as np
import torch
vectors = np.load("/home/chenhaojun/turbozero_torch/24D_196560_1.npy")
board = np.load("/data/haojun/max_board_iter10_replay10/max_board_5.npy")
vectors.shape

(196560, 24)

In [2]:
selected_vectors = vectors[board]
selected_vectors = torch.tensor(selected_vectors, dtype=torch.float16, device='cuda')

In [3]:
valid_cos_arr = torch.tensor([0.25, 0, -1, -0.25, 1], dtype=torch.float16, device='cuda')

In [4]:
def find_valid_rows(cos_matrix, valid_cos_arr, tolerance=1e-2):
    # 使用广播和向量化操作
    diff = torch.abs(cos_matrix.unsqueeze(-1) - valid_cos_arr)
    is_valid = torch.any(diff < tolerance, dim=-1)
    return torch.all(is_valid, dim=-1)

In [5]:
unselected_vectors = vectors[list(range(10))]
unselected_vectors = torch.tensor(unselected_vectors, dtype=torch.float16, device='cuda')

In [6]:
vectors = torch.concat([selected_vectors, unselected_vectors], dim=0)

In [7]:
selected_indices = torch.arange(selected_vectors.shape[0], device='cuda')
unselected_indices = torch.arange(selected_vectors.shape[0], selected_vectors.shape[0]+unselected_vectors.shape[0], device='cuda')

In [8]:
cos_matrix = unselected_vectors @ selected_vectors.T
valid_rows = find_valid_rows(cos_matrix, valid_cos_arr)

In [9]:
valid_rows

tensor([ True, False,  True,  True,  True, False,  True,  True,  True,  True],
       device='cuda:0')

In [10]:
valid_unselected_vectors = unselected_vectors[valid_rows]
valid_unselected_indices = unselected_indices[valid_rows]
valid_unselected_vectors.shape, valid_unselected_indices.shape

(torch.Size([8, 24]), torch.Size([8]))

In [23]:
valid_unselected_indices

tensor([ 5,  7,  8,  9, 11, 12, 13, 14], device='cuda:0')

In [25]:
unselected_indices

tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14], device='cuda:0')

In [26]:
remain_cos_matrix = valid_unselected_vectors @ unselected_vectors.T
# remain_cos_matrix = valid_unselected_vectors @ selected_vectors.T
remain_cos_matrix.shape

torch.Size([8, 10])

In [12]:
torch.sum(torch.isclose(remain_cos_matrix, torch.tensor(0.25, device='cuda', dtype=torch.float16)), dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [13]:
torch.sum(torch.isclose(remain_cos_matrix, torch.tensor(-0.25, device='cuda', dtype=torch.float16)), dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [14]:
torch.sum(torch.isclose(remain_cos_matrix, torch.tensor(0, device='cuda', dtype=torch.float16)), dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [15]:
torch.sum(torch.isclose(remain_cos_matrix, torch.tensor(-1, device='cuda', dtype=torch.float16)), dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [16]:
def count_valid_nums(remain_cos_matrix):
    return torch.sum(torch.isclose(remain_cos_matrix, torch.tensor(0.25, device='cuda', dtype=torch.float16)), dim=1) + torch.sum(torch.isclose(remain_cos_matrix, torch.tensor(-0.25, device='cuda', dtype=torch.float16)), dim=1) + torch.sum(torch.isclose(remain_cos_matrix, torch.tensor(0, device='cuda', dtype=torch.float16)), dim=1) + torch.sum(torch.isclose(remain_cos_matrix, torch.tensor(-1, device='cuda', dtype=torch.float16)), dim=1)

In [17]:
count = count_valid_nums(remain_cos_matrix)
count


tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [27]:
sorted_indices = torch.argsort(count, descending=True)
sorted_indices

tensor([7, 6, 4, 5, 1, 0, 2, 3], device='cuda:0')

: 

In [19]:
valid_unselected_indices[sorted_indices]

tensor([14, 13, 11, 12,  7,  5,  8,  9], device='cuda:0')

In [20]:
torch.all(find_valid_rows(vectors[valid_unselected_indices[sorted_indices]] @ selected_vectors.T, valid_cos_arr))

tensor(True, device='cuda:0')

In [21]:
count.shape

torch.Size([8])

In [22]:
remain_cos_matrix.shape

torch.Size([8, 10])